# Requirements and Data download

## Import Required libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, \
    classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import pickle
import os

os.environ["KERAS_BACKEND"] = "torch"
import re
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gdown
import shutil


## Use GPU and Cuda

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [4]:
with open(f'Rhythm_Encoder.pkl', 'rb') as file:
    rhythm_encoder = pickle.load(file)


C:\Users\KASRA\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
classes = rhythm_encoder.classes_

# Download Model and Tokenizer
## We use gpt2-fa for Model and Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")
tokenizer.add_special_tokens({'pad_token': '<pad>'})

model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/gpt2-fa",
                                                           num_labels=
                                                           len(classes))

model.config.pad_token_id = tokenizer.pad_token_id

C:\Users\KASRA\AppData\Roaming\Python\Python310\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/gpt2-fa and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
len(tokenizer)

42001

In [8]:
tokenizer

GPT2TokenizerFast(name_or_path='HooshvareLab/gpt2-fa', vocab_size=42000, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("<|start

In [9]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(42001, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=10, bias=False)
)

In [10]:
model.config

GPT2Config {
  "_name_or_path": "HooshvareLab/gpt2-fa",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 5,
  "embd_pdrop": 0.1,
  "eos_token_id": 5,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 1,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "s

# Load Trained Model

## You can request to my email to get the dataset
## kasraworkcolab4@gmail.com

In [32]:
model.load_state_dict(torch.load(f'verse_best_model.pt', weights_only=True))

<All keys matched successfully>

# Create PersianPoemDataset

In [11]:
class PersianPoemDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=32):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        encoding = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=20, padding="max_length")

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {'input_ids': input_ids, 'attention_mask': attention_mask}


# Define Input verse_text

In [107]:
verse_texts = [
    "بیا تا گل برافشانیم و می در ساغر اندازیم",
    "فلک را سقف بشکافیم و طرحی نو دراندازیم",

    "اگر غم لشکر انگیزد که خون عاشقان ریزد",
    "من و ساقی به هم تازیم و بنیادش براندازیم",

    "نذر کردم گر از این غم به درآیم روزی",
    "تا درِ میکده شادان و غزل خوان بروم",

    "چنان کاهل بدم کان را نگویم",
    "چو دیدم روی تو مردانه گشتم",

    "فسانه عاشقان خواندم شب و روز",
    "کنون در عشق تو افسانه گشتم",


]
verse_true_labels = [
    "مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)",
    "مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)",

    "مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)",
    "مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)",

    "فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف)",
    "فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف)",

    "مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن دوبیتی)",
    "مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن دوبیتی)",

    "مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن دوبیتی)",
    "مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن دوبیتی)",


]

In [108]:
test_verses_dataset = PersianPoemDataset(verse_texts, tokenizer)
test_verses_loader = DataLoader(test_verses_dataset, batch_size=1)

In [109]:
test_verses_dataset[0]['input_ids']

tensor([ 5483,   399,   953, 27906,   549,   352,   293,   310,   298,   824,
         6555,  6427,   352,     1,     1,     1,     1,     1,     1,     1])

# Make predictions

In [110]:
def prediction(model, data_loader):
    model.to(device)
    model.eval()

    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())

    return all_preds

## Make Predictions on test_loader

In [111]:
test_preds = prediction(model, test_verses_loader)

In [112]:
len(test_verses_loader) * 1, len(test_preds)

(10, 10)

In [113]:
test_preds

[7, 7, 7, 7, 2, 2, 6, 6, 6, 6]

In [114]:
test_preds_labels = rhythm_encoder.inverse_transform(test_preds)

In [115]:
results_df = pd.DataFrame({
    'Verse': verse_texts,
    'Rhythm code': test_preds,
    'Predicted Rhythm Label': test_preds_labels,
    'True Label': verse_true_labels
})


In [116]:
results_df[:]

,Verse,Rhythm code,Predicted Rhythm Label,True Label
0,بیا تا گل برافشانیم و می در ساغر اندازیم,7,مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم),مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)
1,فلک را سقف بشکافیم و طرحی نو دراندازیم,7,مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم),مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)
2,اگر غم لشکر انگیزد که خون عاشقان ریزد,7,مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم),مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)
3,من و ساقی به هم تازیم و بنیادش براندازیم,7,مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم),مفاعیلن مفاعیلن مفاعیلن مفاعیلن (هزج مثمن سالم)
4,نذر کردم گر از این غم به درآیم روزی,2,فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف),فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف)
5,تا درِ میکده شادان و غزل خوان بروم,2,فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف),فعلاتن فعلاتن فعلاتن فعلن (رمل مثمن مخبون محذوف)
6,چنان کاهل بدم کان را نگویم,6,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...
7,چو دیدم روی تو مردانه گشتم,6,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...
8,فسانه عاشقان خواندم شب و روز,6,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...
9,کنون در عشق تو افسانه گشتم,6,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...,مفاعیلن مفاعیلن فعولن (هزج مسدس محذوف یا وزن د...
